In [1]:
import csv
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from random import randint
from tqdm import tqdm
import cv2
from sklearn import datasets, metrics, preprocessing, decomposition
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.feature_extraction import DictVectorizer

X_labels = pd.read_csv('dataset/labels.csv')

sample_submission = pd.read_csv('dataset/sample_submission.csv')
y_data_names = sample_submission['id']

train_data_length = len(X_labels)
test_data_length = len(y_data_names)
breed = set(X_labels['breed'])

class_length = len(breed)
class_to_num = dict(zip(breed, range(class_length)))
num_to_class = dict(zip(range(class_length), breed))

width = 299

X_train = np.zeros((train_data_length, width, width, 3),dtype=np.uint8)
X_test = np.zeros((test_data_length, width, width, 3),dtype=np.uint8)

y_train = np.zeros((train_data_length, class_length),dtype=np.uint8)


X_test_flat = np.zeros((test_data_length, width*width*3),dtype=np.uint8)
X_train_flat = np.zeros((train_data_length, width*width*3),dtype=np.uint8)


In [2]:
for i in tqdm(range(train_data_length)):   
    image = cv2.imread('dataset/train/{}.jpg'.format(X_labels['id'][i]))
    resized = cv2.resize(image,(width, width))
    #gray_image = cv2.cvtColor(resized, cv2.COLOR_BGR2GRAY)
    xarr = resized
    np.squeeze(np.array(resized).astype(np.float32))        
    
    flat_arr = xarr.ravel()

    #X_train[i] = xarr
    X_test_flat[i] = flat_arr
    
    y_train[i][class_to_num[X_labels['breed'][i]]] = 1


100%|██████████| 10222/10222 [00:54<00:00, 187.43it/s]


In [ ]:
for i in tqdm(range(train_data_length)):   
    image = cv2.imread('dataset/test/{}.jpg'.format(y_data_names[i]))
    resized = cv2.resize(image,(width, width))
    #gray_image = cv2.cvtColor(resized, cv2.COLOR_BGR2GRAY)
    xarr = resized
    np.squeeze(np.array(resized).astype(np.float32))        
    
    flat_arr = xarr.ravel()
   
    #X_test[i] = xarr
    X_test_flat[i] = flat_arr
    

100%|██████████| 10222/10222 [00:52<00:00, 195.22it/s]


In [ ]:
X_train_flat = X_train_flat.astype(int)
X_test_flat = X_test_flat.astype(int)

pca = decomposition.PCA(n_components=int(len(X_train_flat[0])), svd_solver='randomized')
pca.fit(X_test_flat[:int(len(X_train_flat)//5)])

for i in tqdm(range(len(X_train_flat))):
    pca_arr = pca.transform(X_train_flat[i].reshape(1, -1))
    X_train_flat[i] = pca_arr
    
for i in tqdm(range(len(X_test_flat))):
    pca_arr = pca.transform(X_test_flat[i].reshape(1, -1))
    X_test_flat[i] = pca_arr


In [ ]:
y_train_categorical = [None] * train_data_length
y_train_categorical = np.array(y_train_categorical)

for i in range(len(y_train)):
    y_train_categorical[i] = X_labels['breed'][i]

In [ ]:
le = preprocessing.LabelEncoder()
le.fit(X_labels['breed'])

print(y_train_categorical[1])
y_train_categorical = le.transform(y_train_categorical) 
print(y_train_categorical[1])

In [ ]:
#svm_classifier = SVC()


#param_grid = {'gamma': [10, 1, 0.1, 0.01], 
#                'C': [1, 10, 100, 1000],
#                'kernel': ['rbf', 'linear'],
#                'probability': [True],
#                'verbose': [True]}
        
#grid = GridSearchCV(estimator=svm_classifier, param_grid=param_grid, verbose = 10, n_jobs=-1)      
#grid.fit(X_train_flat, y_train_categorical)

#print(grid)
#print(grid.best_estimator_) 
        
#file = open('SVM_params.txt', 'a')
#file.write(str(grid))
#file.write('\n')
#file.write(str(grid.best_estimator_))
#file.write('\n')
#file.close() 
                
#svm_classifier = grid.best_estimator_

In [ ]:
svm_classifier = SVC(kernel='linear', C=2.5, gamma=5, probability=True, verbose=True)

print('START FITTING')
svm_classifier.fit(X_train_flat, y_train_categorical)
print('END FITTING')

In [ ]:
#print('START PREDICTING')
#prediction = svm_classifier.predict(X_test_flat)
#print('END PREDICTING')

In [ ]:
print('START PREDICTING PROBABILITY')
probability_result = svm_classifier.predict_proba(X_test_flat)
print('END PREDICTING PROBABILITY')

In [ ]:
import csv

filename = 'submission_nogray.csv'

with open(filename, 'wt') as csvfile:
    csvfile.truncate()
    writer = csv.writer(csvfile, delimiter=',')
    label_names = []
    
    label_names.append('id')
    for b in breed:
        label_names.append(b)
        
    writer.writerow([label for label in label_names])
        
    result_row = []    
    for i in tqdm(range(len(probability_result))):
        result_row.append(y_data_names[i])
        
        for result in probability_result[i]:
            result_row.append(result)
        
        writer.writerow([result for result in result_row])
        
        del result_row[:]